In [9]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import numpy as np
import json
import copy
import folium
import branca
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import linkage, fcluster

import panel as pn
import networkx as nx
import geopandas as gpd
import shapely
from shapely.geometry import LineString, MultiLineString, shape
import shapely.wkt
from matplotlib import cm, colors

## Grafo de conexion entre barrios

In [15]:
from dash import Dash, dcc, html, Input, Output
import geopandas as gpd
import pandas as pd
import shapely.wkt
import networkx as nx
import plotly.graph_objects as go

# Cargar datos
barrios_gdf = gpd.read_file('data/barris.geojson')
bus_lines_df = pd.read_csv("data/linies_bus.csv")

# Convertir WKT a geometría
def safe_load_geometry(wkt_str):
    try:
        return shapely.wkt.loads(wkt_str)
    except Exception:
        return None

bus_lines_df["geometry"] = bus_lines_df["GEOMETRY"].apply(safe_load_geometry)
bus_lines_gdf = gpd.GeoDataFrame(bus_lines_df, geometry="geometry", crs="EPSG:4326")

# Construir grafo
G = nx.Graph()
for idx, row in barrios_gdf.iterrows():
    barrio_name = row["NOM"]
    G.add_node(barrio_name, geometry=row.geometry)

for line in bus_lines_gdf.itertuples():
    if not line.geometry:
        continue
    touched_barrios = set()
    for idx, barrio in barrios_gdf.iterrows():
        if line.geometry.intersects(barrio.geometry):
            touched_barrios.add(barrio["NOM"])
    touched_barrios = list(touched_barrios)
    for i in range(len(touched_barrios)):
        for j in range(i + 1, len(touched_barrios)):
            b1, b2 = touched_barrios[i], touched_barrios[j]
            if G.has_edge(b1, b2):
                G[b1][b2]["weight"] += 1
            else:
                G.add_edge(b1, b2, weight=1)

# Posiciones de nodos
pos = {node: (geom.centroid.x, geom.centroid.y) for node, geom in nx.get_node_attributes(G, 'geometry').items()}
nx.set_node_attributes(G, pos, 'pos')

node_data = [{'id': n, 'x': x, 'y': y} for n, (x, y) in pos.items()]

# App Dash
app = Dash(__name__)
app.layout = html.Div([
    html.H2("Grafo de conexiones entre barrios de Barcelona (líneas de bus)"),
    dcc.Dropdown([n['id'] for n in node_data], id="barrio-dropdown", placeholder="Selecciona un barrio..."),
    dcc.Graph(id="graph")
])

@app.callback(
    Output("graph", "figure"),
    Input("barrio-dropdown", "value")
)
def update_graph(selected_node):
    edge_traces = []
    for u, v, data in G.edges(data=True):
        x0, y0 = G.nodes[u]['pos']
        x1, y1 = G.nodes[v]['pos']
        edge_traces.append(go.Scatter(
            x=[x0, x1, None],
            y=[y0, y1, None],
            line=dict(width=data['weight'], color='red' if selected_node in (u, v) else 'lightgray'),
            mode='lines',
            opacity=1.0 if selected_node in (u, v) else 0.2,
            hoverinfo='text',
            text=[f"{u} ↔ {v}: {data['weight']} líneas"] * 3,
            showlegend=False
        ))

    node_trace = go.Scatter(
        x=[n['x'] for n in node_data],
        y=[n['y'] for n in node_data],
        mode='markers+text',
        text=[n['id'] for n in node_data],
        textposition="top center",
        hoverinfo='text',
        marker=dict(size=10, color=[
            'orange' if n['id'] == selected_node else 'skyblue' for n in node_data
        ], line_width=2),
        showlegend=False
    )

    fig = go.Figure(data=edge_traces + [node_trace])
    fig.update_layout(
        title="Conexiones entre barrios por líneas de bus",
        margin=dict(b=20, l=5, r=5, t=40),
        hovermode='closest',
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
    )
    return fig

if __name__ == '__main__':
    app.run(debug=True, port=8050)


In [ ]:
# Dashboard
pn.extension()

tabs = pn.Tabs(
    ("Análisis de los datos", pn.pane.Matplotlib(sns.pairplot(df[cols], diag_kind='kde', hue=None, height=1, aspect=1, plot_kws={'s': 10}).figure, tight=True)),
    ("Mapa de Puntos", pn.pane.plot.Folium(map_puntos, width=700, height=500)),
    ("Mapa de Clusters", pn.pane.plot.Folium(map_cluster, width=700, height=500)),
    ("Mapa H3", pn.pane.plot.Folium(map_h3, width=700, height=500)),
    ("Mapa rutas finales", pn.pane.plot.Folium(map_obj, width=700, height=500))
)

tabs.servable()